In [ ]:
import pathlib

import numpy as np
import pandas as pd

import pydicom

import pymedphys._dicom.dose as dcmdose

In [ ]:
root = pathlib.Path(r'P:\RT\sheridan\ARPANSA Audit 2020\IMRT Cases 6 to 8 Dicom Files')
total_dose_paths = list(root.glob('*_Dose.dcm'))

In [ ]:
cases = [
    path.name[:-9]
    for path in total_dose_paths
]

cases

In [ ]:
total_dose_paths_by_case = {
    case: path for case, path in zip(cases, total_dose_paths)
}

total_dose_paths_by_case

In [ ]:
dicom_doses = {
    case: pydicom.read_file(total_dose_paths_by_case[case], force=True)
    for case in cases
}

In [ ]:
plan_paths_by_case = {
    case: root.joinpath(f"{case}.dcm")
    for case in cases
}

plan_paths_by_case

In [ ]:
dicom_structure = pydicom.read_file(root.joinpath('SUP3018v1_StrctrSets.dcm'), force=True)

In [ ]:
structure_names = ['11', '12', '13', '14', '15', '16', '17', '18']

In [ ]:
mean_doses = {}

for case in cases:
    mean_doses[case] = {}
    for structure_name in structure_names:
        
        mask = dcmdose.get_dose_grid_structure_mask(structure_name, dicom_structure, dicom_doses[case])
#         print(mask.shape)
        new_mask = np.swapaxes(mask, 0, 1)
        new_mask = np.swapaxes(new_mask, 0, 2)
#         print(new_mask.shape)
        
        dose = dcmdose.dose_from_dataset(dicom_doses[case])
        
        mean_doses[case][structure_name] = np.mean(dose[new_mask])

mean_doses

In [ ]:
pd.DataFrame.from_dict(mean_doses).to_csv('test.csv')